In [1]:
# Cell 1: Environment & path setup

import os
import sys

# Make sure "src" is importable
PROJECT_ROOT = os.path.abspath(".")
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

print("Project root:", PROJECT_ROOT)


Project root: /Users/yk2047/Documents/GitHub/hathor/langchein


In [2]:
# Cell 2: Import agents, config, utils, pipeline

from src.agents import (
    LiteratureAgent,
    CriticAgent,
    ReasonerAgent,
    CoderAgent,
    RunnerAgent,
)
from src.pipeline.multi_agent_pipeline import MultiAgentPipeline
from src.config import (
    LITERATURE_MODEL,
    CRITIC_LITERATURE_MODEL,
    REASONER_MODEL,
    CRITIC_MODEL,
    CODER_MODEL,
    BINARY_FILE_PATH,
    IDEAS_DIR,
    PREVIOUS_CODES_DIR,
    OUTPUT_FIG_PATH,
    FINAL_CODE_PATH,
    DATA_DESCRIPTION_PATH,
)
from src.utils import (
    read_codes_from_folder,
    save_answer_to_file,
    save_code_to_file,
)

print("Binary file:", BINARY_FILE_PATH)
print("Description file:", DATA_DESCRIPTION_PATH)


Binary file: /Users/yk2047/Documents/GitHub/hathor/dataset_examples/halo_3517_gas.bin
Description file: ./plotting_codes/Example.txt


In [3]:
# Cell 3: Instantiate agents (same as pipeline __init__)

literature_agent = LiteratureAgent(LITERATURE_MODEL)
critic_agent = CriticAgent(CRITIC_LITERATURE_MODEL)
reasoner_agent = ReasonerAgent(REASONER_MODEL)
coder_agent = CoderAgent(CODER_MODEL)
runner_agent = RunnerAgent(coder_agent)

print("Agents instantiated ✅")


Agents instantiated ✅


In [4]:
# Cell 4: Test literature_agent.generate_hypotheses()

literature_task = """
I want to do plots for MEGATRON cutout data of gas cells in a halo. It's stored in a binary file. 
The data contains positions (x,y,z), levels, ne (electron number density), dx (cell size), and other 
features for each gas cell.

I want to create 2D images of parameters projected along the z-axis to learn something interesting 
about galaxy clusters.

Please propose interesting hypotheses about galaxy clusters that can be tested through visualization 
and analysis of this data.
"""

num_hypotheses = 3

hypotheses = literature_agent.generate_hypotheses(
    task=literature_task,
    num_hypotheses=num_hypotheses,
)

print("=== Hypotheses ===")
print(hypotheses)


=== Hypotheses ===
**Hypothesis 1 – Radial “cool‑core” imprint in the projected electron‑density map**  
*What to look for:* When the ne field is summed (or column‑averaged) along the z‑axis, the resulting 2‑D image should display a steep central excess of surface electron‑density that falls off roughly as a β‑model ( Σₑ(r) ∝ [1+(r/rₚ)²]^{–3β+0.5} ). In a genuine cool‑core cluster this central peak will be sharply confined (rₚ ≲ 100 kpc) and roughly circular, whereas a non‑cool‑core system will show a flatter core and more irregular isodensity contours. By fitting concentric annuli to the projected map and extracting a radial profile, you can test whether the core slope, core radius, and β‑parameter match the cool‑core expectations and how they vary with halo mass or redshift within the MEGATRON sample.

**Hypothesis 2 – Gas‑clumping factor rises toward the virial boundary**  
*What to look for:* The “clumping factor” C ≡ ⟨nₑ²⟩/⟨nₑ⟩² can be approximated from the 2‑D projection by compa

In [5]:
# Cell 5: Test critic_agent.review_literature()

refined_hypotheses = critic_agent.review_literature(hypotheses)

print("=== Refined hypotheses ===")
print(refined_hypotheses)


=== Refined hypotheses ===
Below is a critical appraisal of the three ideas that were put forward, followed by a trimmed‑down set of **realistic, testable hypotheses** that can be answered directly with the 2‑D maps you will generate from the Megatron cut‑outs.  I have kept the spirit of each original claim, but removed the parts that are either impossible to measure reliably or that would give a misleading answer, and I have added the quantitative diagnostics that make the tests concrete.

---

## 1.  Radial “cool‑core” imprint in the projected electron‑density map  

### What is problematic?  

| Issue | Why it matters | Remedy |
|------|----------------|--------|
| **Projection‑induced dilution** – a steep 3‑D core becomes much shallower once integrated along a 2‑Mpc line of sight. | A β‑model fit to the *projected* profile will systematically underestimate the true central slope and core radius, especially for halos that are not perfectly spherical. | Use **de‑projected** profiles 

In [6]:
# Cell 6: Test reasoner_agent.create_pairs()

previous_codes = read_codes_from_folder(PREVIOUS_CODES_DIR)
print(f"Loaded {len(previous_codes)} previous code snippets.\n")

pairs = reasoner_agent.create_pairs(refined_hypotheses, previous_codes)

print("=== Initial hypothesis–plan pairs ===")
print(pairs)


Loaded 39018 previous code snippets.

=== Initial hypothesis–plan pairs ===
**Overall workflow (common to all three hypotheses)**  

1. **Load the Megatron cut‑outs** – read the binary file with the `read_megatron_cutout` routine, keep at least the columns  
   `x, y, z, dx, ne, T, level, H_I, H_II, He_II, He_III` (and any metal columns you may need for later sanity checks).  
2. **Choose a physical scaling** – convert the raw “code units” to physical kpc (or proper Mpc) using the supplied redshift, box size and the cosmology factor `h`.  Store a per‑cell physical size `Δ = 10**dx`.  
3. **Select a line‑of‑sight** – produce two orthogonal projections (`xy` and `xz` are a good default) so that orientation effects can be diagnosed.  
4. **Generate the basic 2‑D maps** – use the `make_image` routine (or an equivalent CIC/DP‑sampling) to obtain, at a fixed physical pixel scale (≈ 5 kpc is a safe baseline), the following projected fields:  
   * **Σₙₑ** – surface electron‑density (weight = 

In [7]:
# Cell 7: One round of critic → reasoner refinement

critique = critic_agent.review_pairs(pairs)
print("=== Critic feedback on pairs ===")
print(critique)

if "plan is ok" in critique.lower() or "plan ok" in critique.lower():
    print("\nCritic says plan is OK ✅")
    final_pair = pairs
else:
    print("\nCritic eliminated or modified pairs, sending back to Reasoner...")
    refinement_task = (
        f"Previous hypothesis-plan pairs:\n{critique}\n\n"
        "Please review and refine these pairs based on the feedback."
    )
    refined_pairs = reasoner_agent.create_pairs(refinement_task, previous_codes)
    final_pair = refined_pairs

print("\n=== Final pair candidate ===")
print(final_pair)


=== Critic feedback on pairs ===
**Review of the three hypothesis – plan pairs**

| Hypothesis | Overall merit of the proposed plan |
|------------|-----------------------------------|
| **H1‑R** “Cool‑core concentration” | solid – uses well‑tested surface‑density and entropy proxies, elliptical annuli, and several sanity‑checks (projection, resolution, mass/redshift trends). |
| **H2‑R** “Projected clumping gradient” | weakest – the clumping estimator **Cₚᵣₒⱼ** is extremely sensitive to (i) pixel‑scale shot noise, (ii) the arbitrary choice of smoothing, (iii) the projection geometry, and (iv) the way the effective number of cells *Nₑ𝚏𝚏* is estimated.  Even after the proposed corrections the resulting radial slope **α** can be dominated by artefacts, making the hypothesis difficult to test robustly with the available data. |
| **H3‑R** “Refinement‑gradient merger tracer” | strong – exploits the AMR level‑excess field, a quantity that directly encodes gradient‑driven refinement.  The qu

In [8]:
# Cell 8: Build coder_instructions by hand and inspect

code_developer_prompt = f"""
I need to create visualization code for MEGATRON cutout data of gas cells in a halo. 
The binary file is at path: "{BINARY_FILE_PATH}"

The data contains positions (x,y,z), levels, ne (electron number density), dx (cell size), and other 
features for each gas cell.

Create Python code to implement the hypothesis and plan. The image should have a resolution of 512x512 
pixels and cover a box size of 20 Mpc at redshift z=0.5. The levels range from 12 to 18.
"""

coder_instructions = (
    f"{code_developer_prompt}\n\n"
    f"The MEGATRON gas binary file is located at: {BINARY_FILE_PATH}\n\n"
    f"There is also a separate data-description text file at:\n"
    f"{DATA_DESCRIPTION_PATH}\n\n"
    "In your Python code, load this description file (e.g., with "
    "numpy.loadtxt or similar) and use it to inform column meanings, "
    "units, and/or figure labelling. Do NOT ignore this file.\n\n"
    f"Hypothesis and Plan to implement:\n{final_pair}\n\n"
    "If you use functions like read_megatron_cutout(), write it in the code, "
    "don't just import it elsewhere.\n\n"
    f"Save the final plot as '{OUTPUT_FIG_PATH}'."
)

if previous_codes:
    coder_instructions += f"\n\nReference codes:\n{previous_codes}"

print(coder_instructions[:1500])  # preview first part



I need to create visualization code for MEGATRON cutout data of gas cells in a halo. 
The binary file is at path: "/Users/yk2047/Documents/GitHub/hathor/dataset_examples/halo_3517_gas.bin"

The data contains positions (x,y,z), levels, ne (electron number density), dx (cell size), and other 
features for each gas cell.

Create Python code to implement the hypothesis and plan. The image should have a resolution of 512x512 
pixels and cover a box size of 20 Mpc at redshift z=0.5. The levels range from 12 to 18.


The MEGATRON gas binary file is located at: /Users/yk2047/Documents/GitHub/hathor/dataset_examples/halo_3517_gas.bin

There is also a separate data-description text file at:
./plotting_codes/Example.txt

In your Python code, load this description file (e.g., with numpy.loadtxt or similar) and use it to inform column meanings, units, and/or figure labelling. Do NOT ignore this file.

Hypothesis and Plan to implement:
Below are the **refined, end‑to‑end plans** for the two remaini

In [ ]:
# Cell 9: Generate code with CoderAgent
import numpy as np
coder_instructions = np.loadtxt(DATA_DESCRIPTION_PATH, dtype=str)
generated_code = coder_agent.generate_code(coder_instructions)

print("=== Generated code (truncated) ===")
print(generated_code[:3000])


=== Generated code (truncated) ===
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from matplotlib.colors import LogNorm
from scipy.io import FortranFile

# ----------------------------------------------------------------------
# Helper to read description file (used for labeling only)
def load_description(desc_path):
    desc = {}
    if os.path.exists(desc_path):
        with open(desc_path, "r") as f:
            for line in f:
                line = line.strip()
                if not line or line.startswith("#"):
                    continue
                parts = line.split()
                if len(parts) >= 2:
                    col = parts[0].strip()
                    unit = parts[-1].strip("[]")
                    desc[col] = unit
    return desc

# ----------------------------------------------------------------------
# Minimal Megatron cutout reader (fallback if external utils not available)
def rea

In [ ]:
# Cell 10: Run the generated code through RunnerAgent

final_code = runner_agent.run_with_debug(generated_code)

print("\n=== Final code after runner debugging (truncated) ===")
print(final_code[:3000])
# Cell 11: Save outputs to files
# save_answer_to_file(OUTPUT_FIG_PATH, "Figure generated by the pipeline.")




STEP 4: RUNNER AGENT (Iteration 1)


STEP 4: RUNNER AGENT (Iteration 2)


STEP 4: RUNNER AGENT (Iteration 3)


🎉 SUCCESS: Code executed successfully!

=== Final code after runner debugging (truncated) ===
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from matplotlib.colors import LogNorm
from scipy.io import FortranFile

def load_description(desc_path):
    import os
    desc = {}
    if os.path.exists(desc_path):
        with open(desc_path, "r") as f:
            for line in f:
                line = line.strip()
                if not line or line.startswith("#"):
                    continue
                parts = line.split()
                if len(parts) >= 2:
                    col = parts[0].strip()
                    unit = parts[-1].strip("[]")
                    desc[col] = unit
    return desc

def read_megatron_cutout_local(ff):
    import numpy as np
    import pandas as pd
    header = [
        "redshi

OSError: [Errno 63] File name too long: 'import numpy as np\nimport pandas as pd\nimport matplotlib.pyplot as plt\nfrom matplotlib.patches import Ellipse\nfrom matplotlib.colors import LogNorm\nfrom scipy.io import FortranFile\n\ndef load_description(desc_path):\n    import os\n    desc = {}\n    if os.path.exists(desc_path):\n        with open(desc_path, "r") as f:\n            for line in f:\n                line = line.strip()\n                if not line or line.startswith("#"):\n                    continue\n                parts = line.split()\n                if len(parts) >= 2:\n                    col = parts[0].strip()\n                    unit = parts[-1].strip("[]")\n                    desc[col] = unit\n    return desc\n\ndef read_megatron_cutout_local(ff):\n    import numpy as np\n    import pandas as pd\n    header = [\n        "redshift","dx","x","y","z","vx","vy","vz","nH","T","P",\n        "nFe","nO","nN","nMg","nNe","nSi","nCa","nC","nS","nCO",\n        "O_I","O_II",

In [12]:
save_code_to_file(final_code, "./outputs/executed_codes/test.py")


In [ ]:
# Cell 11: Full pipeline run (same as main())

pipeline = MultiAgentPipeline()

pipeline.run(
    literature_task=literature_task,
    code_developer_prompt=code_developer_prompt,
    num_hypotheses=3,
    max_refinement_iterations=2,
)


In [ ]:
# Cell 12: Inspect output files

print("IDEAS_DIR contents:", os.listdir(IDEAS_DIR))
print("Final code path:", FINAL_CODE_PATH)
print("Output figure path:", OUTPUT_FIG_PATH)

if os.path.exists(FINAL_CODE_PATH):
    with open(FINAL_CODE_PATH, "r") as f:
        print("\n=== Final code (first 500 chars) ===")
        print(f.read()[:500])
